# Perturbing an existing ACE file

In this example, we will take an existing incident neutron ACE file and modify a single reaction's cross section data by 250%.

The first thing we will need is the name of the ACE file that we want to perturb, the energy range over which to perform the perturbation, the MT number of the reaction we wish to change and the scaling factor we wish to apply. For the energy values, we have chosen energy values that are present in the common energy grid (for the purpose of simplicity).

In [ ]:
filename = '2003.710nc'
perturbedfilename = '2003.perturbed.710nc'
mt = 103
lower = 1e-8 # 1e-2 eV
upper = 1.2  # 1.2 MeV
scale = 2.5

We will now load the ACE file and retrieve the reaction index for the reaction that we wish to perturb. The reaction index can be retrieved from the MTR block (containing the list of MT numbers available in the ACE file). The MTR block interface provides a function that allows us to check for the presence of a given MT number before asking for the actual index. When the requested MT number is not present, the index function will throw an exception. The index returned will be a 1-based index.

In [ ]:
import ACEtk

he3 = ACEtk.ContinuousEnergyTable.from_file( filename )
index = he3.MTR.index( mt )

print( he3.MTR.reaction_numbers.to_list() )
print( index )

Note: Since we need to perturb an energy range, we are required to add energy points to the common energy grid and therefore modify the all the data that depends on this (i.e. the ESZ block with the common energy grid, the cross sections stored in the SIG block and some of the data associated with the secondary photons and the secondary particle types). For the purpose of this example, we will not do this as this would unnecessarily complicate things.

We will now retrieve the index of the two energy points that constitute the range in which we wish to make the perturbation. The indices returned are 0-based since we use the built-in index() function on a Python list.

In [ ]:
energies = he3.ESZ.energies.to_list()
start = energies.index( lower )
stop = energies.index( upper )

print( start, stop )

We will now retrieve the cross section data for the reaction we wish to modify and generate new cross section data that will replace the old data by scaling the cross section directly. Since we are not adding new energy points, we do not need to add any additional cross section values or update the energy index.

In [ ]:
oldxs = he3.SIG.cross_section_data( index )
newxsvalues = oldxs.cross_sections.to_list()
for i in range( start, stop + 1 ) : newxsvalues[i] *= scale

newxs = ACEtk.CrossSectionData( index = oldxs.energy_index, values = newxsvalues )

Next, we will retrieve all the cross section stored in the SIG block and replace the old cross section data with the newly created data.

In [ ]:
xs = he3.SIG.data
xs[index - 1] = newxs # the data is a python list so we need a 0-based index

We are now ready to create a new ACE file based on the old one. The ACE file contains a lot of internal locators and the constructors of the various components will take care of readjusting the locators if required. Since we did not add any energy points to the common energy grid, the locators will actually not be changed. The original ACE file also contained secondary particle distribution data but for the sake of simnplicity, the example does not cover over the data for the secondary particles except for the incident particle type.

In [ ]:
newhe3 = ACEtk.ContinuousEnergyTable( 
           header = he3.header, z = he3.Z, a = he3.A,
           esz = he3.ESZ, mtr = he3.MTR, lqr = he3.LQR,
           sig = ACEtk.CrossSectionBlock( xs ),
           ang = he3.AND, dlw = he3.DLW )
newhe3.to_file( perturbedfilename )

To illustrate that the new file is now readable, we will now open the created ACE file and extract the modified cross section data to plot it against the old data.

In [ ]:
newhe3 = ACEtk.ContinuousEnergyTable.from_file( perturbedfilename )
index = he3.MTR.index( mt )

print( he3.MTR.reaction_numbers.to_list() )
print( index )

In [ ]:
import matplotlib.pyplot as plot

plot.plot( energies, oldxs.cross_sections, label = 'original' )
plot.plot( energies, newhe3.SIG.cross_section_data( index ).cross_sections, label = 'perturbed' )
plot.xlabel( 'Energy (MeV)' )
plot.ylabel( 'Cross Section (b)' )
plot.title( 'He3 n,p' )
plot.xscale( 'log' )
plot.yscale( 'log' )
plot.legend()
plot.show()